In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
from descriptors import sift_descriptor, brisk_descriptor, keypoint_detector
from hilbert_curve import image_descriptor as hilbert_descriptor
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image

In [2]:
sift_label = []
sift_data = []

brisk_label = []
brisk_data = []

hilbert_label = []
hilbert_data = []



#p_class1 = Path('../dtd/images/bubbly')
#p_class2 = Path('../dtd/images/cracked')

p_class1 = Path('../img/clean-bg/motorbike')
p_class2 = Path('../img/clean-bg/airplane')

# n_images = min(len(list(p_bubbly.glob('*'))), len(list(p_cracked.glob('*'))))
n_images = 100

### Descritores e labels da classe 1

In [3]:
for f in list(p_class1.iterdir())[:n_images]:
    
    if f.suffix in ['.jpg', '.png', '.jpeg']:
    
        #img = cv.imread(str(f))   
        
        img = Image.open(str(f)).convert('L')
        img_array = np.asarray(img)        
        
        keypoints = keypoint_detector(img_array)
        
        sift_desc = sift_descriptor(img_array, keypoints)
        brisk_desc = brisk_descriptor(img_array, keypoints)
        hilbert_desc = hilbert_descriptor(img_array, keypoints)

        sift_data.extend(sift_desc)
        brisk_data.extend(brisk_desc)
        hilbert_data.extend(hilbert_desc)

        sift_label.extend([0]*len(sift_desc)) # 0 => motorbike
        brisk_label.extend([0]*len(brisk_desc))
        hilbert_label.extend([0]*len(hilbert_desc))
        


### Descritores e labels da classe 2

In [4]:
for f in list(p_class2.iterdir())[:n_images]:
    
    if f.suffix in ['.jpg', '.png', '.jpeg']:
    
        #img = cv.imread(str(f)) 
        img = Image.open(str(f)).convert('L')
        img_array = np.asarray(img)        
        
        keypoints = keypoint_detector(img_array)
        
        sift_desc = sift_descriptor(img_array, keypoints)
        brisk_desc = brisk_descriptor(img_array, keypoints)
        hilbert_desc = hilbert_descriptor(img_array, keypoints)

        sift_data.extend(sift_desc)
        brisk_data.extend(brisk_desc)
        hilbert_data.extend(hilbert_desc)

        sift_label.extend([1]*len(sift_desc)) # 1 => airplane
        brisk_label.extend([1]*len(brisk_desc))
        hilbert_label.extend([1]*len(hilbert_desc))
    

In [5]:
sift_desc.shape

(277, 128)

In [6]:
brisk_desc.shape

(277, 64)

In [7]:
hilbert_desc.shape

(233, 6)

In [8]:
sift_label = np.asarray(sift_label)
brisk_label = np.asarray(brisk_label)
hilbert_label = np.asarray(hilbert_label)

### K-Nearest Neighbors usando descritor SIFT

In [9]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from joblib import dump, load

In [10]:
def create_model_report(data, label, print_report=False):
    
    X_train, X_test, y_train, y_test = train_test_split(data, label)
    
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    report = classification_report(y_test, predicted, output_dict=True, target_names=['motorbike', 'airplane'])
    
    if print_report:
        print(classification_report(y_test, predicted, target_names=['motorbike', 'airplane']))
    
    return knn, report
    

### Classification Report - SIFT

In [11]:
knn_sift, report_sift = create_model_report(sift_data, sift_label, print_report=True)

              precision    recall  f1-score   support

   motorbike       0.79      0.75      0.77      4840
    airplane       0.69      0.73      0.71      3569

    accuracy                           0.74      8409
   macro avg       0.74      0.74      0.74      8409
weighted avg       0.75      0.74      0.75      8409



### Classification Report - BRISK

In [12]:
knn_brisk, report_brisk = create_model_report(brisk_data, brisk_label, print_report=True)

              precision    recall  f1-score   support

   motorbike       0.72      0.66      0.69      4815
    airplane       0.59      0.66      0.62      3540

    accuracy                           0.66      8355
   macro avg       0.66      0.66      0.66      8355
weighted avg       0.67      0.66      0.66      8355



### Classification Report - Hilbert Curve

In [13]:
knn_hilbert, report_hilbert = create_model_report(hilbert_data, hilbert_label, print_report=True)

              precision    recall  f1-score   support

   motorbike       0.88      0.85      0.86      4726
    airplane       0.80      0.83      0.81      3373

    accuracy                           0.84      8099
   macro avg       0.84      0.84      0.84      8099
weighted avg       0.84      0.84      0.84      8099



Saving models

In [15]:
dump(knn_sift, '../models/knn_sift.z', compress=3)
dump(knn_brisk, '../models/knn_brisk.z', compress=3)
dump(knn_hilbert, '../models/knn_hilbert.z', compress=3)

['../models/knn_hilbert.z']

### Gerar tabela LaTeX com métricas dos modelos 

In [16]:
import pandas as pd

In [25]:
report_all = {'SIFT':{}, 'BRISK':{}, 'Hilbert':{}}

report_all['SIFT']['accuracy'] = np.round(report_sift['accuracy'], 2)
report_all['BRISK']['accuracy'] = np.round(report_sift['accuracy'], 2)
report_all['Hilbert']['accuracy'] = np.round(report_hilbert['accuracy'], 2)


for key in ['precision', 'recall', 'f1-score']:
    report_all['SIFT'][key] = np.round(report_sift['weighted avg'][key], 2)
    report_all['BRISK'][key] = np.round(report_brisk['weighted avg'][key], 2)
    report_all['Hilbert'][key] = np.round(report_hilbert['weighted avg'][key], 2)
    

In [26]:
df_report_all = pd.DataFrame(report_all).transpose()

In [27]:
latex_report = df_report_all.to_latex(caption = "Comparação das métricas dos modelos KNN utilizando descritores SIFT e de Hilbert")

In [28]:
print(latex_report)

\begin{table}
\centering
\caption{Comparação das métricas dos modelos KNN utilizando descritores SIFT e de Hilbert}
\begin{tabular}{lrrrr}
\toprule
{} &  accuracy &  precision &  recall &  f1-score \\
\midrule
SIFT    &      0.74 &       0.75 &    0.74 &      0.75 \\
BRISK   &      0.74 &       0.67 &    0.66 &      0.66 \\
Hilbert &      0.84 &       0.84 &    0.84 &      0.84 \\
\bottomrule
\end{tabular}
\end{table}

